<a href="https://colab.research.google.com/github/sipocz/CabPrice/blob/B20210523/Model_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [86]:
def DataPipeLine(df, startup=True):
    #outlierek 
    if startup:
        df["price_per"]=df.fare/df.surge_multiplier/df.distance
        df.drop(df[df['price_per'] > 73].index, inplace = True)
        #df.drop(df[df.cab_provider=="Lyft"].index, inplace=True)
    
    df["sec"]=(df.time_stamp/1000) % (24*3600)
    seconds_in_day = 24*60*60
    df['sin_time'] = (np.sin(2*np.pi*df.sec/seconds_in_day))
    df['cos_time'] = (np.cos(2*np.pi*df.sec/seconds_in_day))

    sources=pd.get_dummies(df["source"],prefix='source',drop_first=False)
    dest=pd.get_dummies(df["destination"],prefix='dest',drop_first=False)
    cab=pd.get_dummies(df["cab_provider"],prefix='provider',drop_first=False)
    
    a=df.time_stamp
    df["DateTime"]=a.apply( lambda x:pd.Timestamp(x,unit="ms"))
    #df["dow"]=df["DateTime"].dt.dayofweek
    
    #dow=pd.get_dummies(df["dow"],prefix='dow',drop_first=False)


    sources=pd.get_dummies(df["source"],prefix='source',drop_first=False)
    dest=pd.get_dummies(df["destination"],prefix='dest',drop_first=False)
    cab=pd.get_dummies(df["cab_provider"],prefix='provider',drop_first=False)
    cabtype=pd.get_dummies(df["cab_type"],prefix='cab_type',drop_first=False)

    new = pd.DataFrame([df.DateTime, sources.columns, dest.columns])


    Xret = pd.DataFrame([
                    
                    cab['provider_Lyft'],
                    cab['provider_Uber'],
                    sources['source_Back Bay'],sources['source_Beacon Hill'],sources['source_Boston University'],
                    sources['source_Fenway'],sources['source_Financial District'],sources['source_Haymarket Square'],
                    sources['source_North End'],sources['source_North Station'],sources['source_Northeastern University'],
                    sources['source_South Station'],sources['source_Theatre District'],sources['source_West End'],
                     
                    dest['dest_Back Bay'],dest['dest_Beacon Hill'],dest['dest_Boston University'],
                    dest['dest_Fenway'],dest['dest_Financial District'],dest['dest_Haymarket Square'],
                    dest['dest_North End'],dest['dest_North Station'],dest['dest_Northeastern University'],
                    dest['dest_South Station'],dest['dest_Theatre District'],dest['dest_West End'],

                    cabtype['cab_type_Black'],cabtype['cab_type_Black SUV'],cabtype['cab_type_Lux'],cabtype['cab_type_Lux Black'],
                    cabtype['cab_type_Lux Black XL'],cabtype['cab_type_Lyft'],cabtype['cab_type_Lyft XL'],cabtype['cab_type_Shared'],
                    cabtype['cab_type_UberPool'],cabtype['cab_type_UberX'],cabtype['cab_type_UberXL'],cabtype['cab_type_WAV'],
                    #dow["dow_0"],dow["dow_1"],dow["dow_2"],



                    df["distance"],   # /10,
                    df["surge_multiplier"], #/3,
                    (df["sin_time"]/2)+0.5,
                    (df["cos_time"]/2)+0.5
                    #df.fare/df["surge_multiplier"]/df["distance"],
                    #df.fare
                    ]).transpose()
    return(Xret)



In [87]:
!wget -N "https://github.com/sipocz/CabPrice/raw/main/models/Lyft_model_1__1.55319__.hdf5"

--2021-05-23 12:17:24--  https://github.com/sipocz/CabPrice/raw/main/models/Lyft_model_1__1.55319__.hdf5
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/CabPrice/main/models/Lyft_model_1__1.55319__.hdf5 [following]
--2021-05-23 12:17:24--  https://raw.githubusercontent.com/sipocz/CabPrice/main/models/Lyft_model_1__1.55319__.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2984400 (2.8M) [application/octet-stream]
Saving to: ‘Lyft_model_1__1.55319__.hdf5’

Lyft_model_1__1.553 100%[===================>]   2.85M  --.-KB/s    in 0.1s    

Last-modified header missing -- time-stamps turned

In [88]:

!wget -N "https://github.com/sipocz/CabPrice/raw/main/models/Uber_model_1__1.90707__.hdf5"

--2021-05-23 12:17:25--  https://github.com/sipocz/CabPrice/raw/main/models/Uber_model_1__1.90707__.hdf5
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/CabPrice/main/models/Uber_model_1__1.90707__.hdf5 [following]
--2021-05-23 12:17:25--  https://raw.githubusercontent.com/sipocz/CabPrice/main/models/Uber_model_1__1.90707__.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2984400 (2.8M) [application/octet-stream]
Saving to: ‘Uber_model_1__1.90707__.hdf5’

Uber_model_1__1.907 100%[===================>]   2.85M  --.-KB/s    in 0.1s    

Last-modified header missing -- time-stamps turned

In [89]:
Lyftmodelname="Lyft_model_1__1.55319__.hdf5"
Lyftmodel=tf.keras.models.load_model(Lyftmodelname)

In [90]:
Ubermodelname="Uber_model_1__1.90707__.hdf5"
Ubermodel=tf.keras.models.load_model(Ubermodelname)

In [91]:
!wget https://github.com/sipocz/CabPrice/raw/B20210507/TEST.csv

--2021-05-23 12:17:26--  https://github.com/sipocz/CabPrice/raw/B20210507/TEST.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/CabPrice/B20210507/TEST.csv [following]
--2021-05-23 12:17:26--  https://raw.githubusercontent.com/sipocz/CabPrice/B20210507/TEST.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1757163 (1.7M) [text/plain]
Saving to: ‘TEST.csv.3’

TEST.csv.3          100%[===================>]   1.67M  --.-KB/s    in 0.07s   

2021-05-23 12:17:27 (22.9 MB/s) - ‘TEST.csv.3’ saved [1757163/1757163]



In [92]:
dft=pd.read_csv("TEST.csv")

In [93]:
dft.head()

,index,time_stamp,cab_provider,source,destination,distance,surge_multiplier,cab_type
0,0,1543364124206,Lyft,Theatre District,South Station,0.59,1.0,Lux
1,1,1543364124206,Lyft,Theatre District,South Station,0.59,1.0,Lux Black XL
2,2,1543364124206,Uber,Financial District,Fenway,6.83,1.0,UberX
3,3,1543364124206,Uber,Beacon Hill,Haymarket Square,1.35,1.0,UberXL
4,4,1543364124206,Lyft,Theatre District,South Station,0.59,1.0,Lyft XL


In [94]:
dft.tail()

,index,time_stamp,cab_provider,source,destination,distance,surge_multiplier,cab_type
24995,24995,1543413683462,Uber,North Station,Fenway,3.05,1.0,UberPool
24996,24996,1543413683462,Uber,North Station,Fenway,3.05,1.0,WAV
24997,24997,1543413683462,Uber,North Station,Fenway,3.05,1.0,Black SUV
24998,24998,1543413683687,Lyft,Boston University,Theatre District,2.96,1.0,Shared
24999,24999,1543413683687,Lyft,Boston University,Theatre District,2.96,1.0,Lux


In [95]:

X_test=DataPipeLine(dft,startup=False)

In [96]:
X_test.head()

,provider_Lyft,provider_Uber,source_Back Bay,source_Beacon Hill,source_Boston University,source_Fenway,source_Financial District,source_Haymarket Square,source_North End,source_North Station,source_Northeastern University,source_South Station,source_Theatre District,source_West End,dest_Back Bay,dest_Beacon Hill,dest_Boston University,dest_Fenway,dest_Financial District,dest_Haymarket Square,dest_North End,dest_North Station,dest_Northeastern University,dest_South Station,dest_Theatre District,dest_West End,cab_type_Black,cab_type_Black SUV,cab_type_Lux,cab_type_Lux Black,cab_type_Lux Black XL,cab_type_Lyft,cab_type_Lyft XL,cab_type_Shared,cab_type_UberPool,cab_type_UberX,cab_type_UberXL,cab_type_WAV,distance,surge_multiplier,sin_time,cos_time
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.59,1.0,0.53358,0.998871
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.59,1.0,0.53358,0.998871
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6.83,1.0,0.53358,0.998871
3,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.35,1.0,0.53358,0.998871
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.59,1.0,0.53358,0.998871


In [97]:
remover=X_test[X_test.provider_Uber==1].index
X_test_Lyft=X_test.drop(remover, inplace=False)

remover=X_test[X_test.provider_Lyft==1].index
X_test_Uber=X_test.drop(remover, inplace=False)





In [98]:
y_test_Lyft=Lyftmodel.predict(X_test_Lyft)


In [99]:
y_test_Uber=Ubermodel.predict(X_test_Uber)

In [100]:
y_test_Uber

array([[16.255634],
       [13.223509],
       [15.929255],
       ...,
       [ 9.694903],
       [10.838896],
       [32.98899 ]], dtype=float32)

In [101]:
df_Uber=pd.DataFrame(y_test_Uber,index=X_test_Uber.index)

In [102]:
df_Uber.head()

,0
2,16.255634
3,13.223509
12,15.929255
21,31.324100
23,11.244592


In [103]:
df_Lyft=pd.DataFrame(y_test_Lyft,index=X_test_Lyft.index)

In [104]:
df_all=df_Lyft.append(df_Uber)

In [105]:
df_all.sort_index(inplace=True)

In [106]:
df_all.head()

,0
0,11.501850
1,26.439692
2,16.255634
3,13.223509
4,10.075735


In [107]:
def round_half(a):
    #return(a)
    import math
    e=math.floor(a)
    m=a-e
    if m>0.85:
        return e+1
    if m<0.15:
        return e
    return a

In [108]:
df_all.columns=["fareX"]
df_all.index.name="index"
df_all["fare"]=df_all["fareX"].apply(lambda x:round_half(x))


In [109]:
#del odf["fare2"] #drop(['algebra', 'chemistry'], axis=1)
df_all.head()

,fareX,fare
index,,
0,11.501850,11.501850
1,26.439692,26.439692
2,16.255634,16.255634
3,13.223509,13.223509
4,10.075735,10.000000


In [110]:
df_all.drop(labels=["fareX"],inplace=True, axis=1)

In [111]:
df_all.to_csv("submission_29.csv")

In [112]:
! head submission_29.csv

index,fare
0,11.501850128173828
1,26.4396915435791
2,16.255634307861328
3,13.223508834838867
4,10.0
5,9.34630012512207
6,5.0
7,7.0
8,28.31490707397461
